In [99]:
import os
import sys

from pyspark.sql.functions import count, col, sum, length


In [2]:
import pandas as pd
from IPython.display import display

# Adjust Pandas display options
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows (optional for large datasets)
pd.set_option('display.width', None)        # Expand display width to fit content


In [3]:
# Add the project root directory (CrashAnalysisProject) to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Adjust if needed
sys.path.append(project_root)

# Verify that the src folder is now accessible
print(sys.path)

['/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/prakhar/Desktop/crash_analysis/venv/lib/python3.8/site-packages', '/home/prakhar/Desktop/crash_analysis']


In [4]:
data_dir = '../data/raw/'

In [5]:
# creating a spark session 
from src.utils.spark_helper import SparkHelper

spark_helper = SparkHelper()

spark = spark_helper.create_spark_session()

print(f"Spark Version: {spark.version}")

25/01/20 01:12:07 WARN Utils: Your hostname, prakhar-Predator-PH315-51 resolves to a loopback address: 127.0.1.1; using 192.168.1.15 instead (on interface wlp0s20f3)
25/01/20 01:12:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/20 01:12:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/20 01:12:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/01/20 01:12:08 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Spark Version: 3.5.4


In [6]:
df_person = spark.read.csv(data_dir+'Primary_Person_use.csv', header=True, inferSchema=True)

In [7]:
df_unit = spark.read.csv(data_dir+'Units_use.csv', header=True, inferSchema=True)

In [8]:
# airbag status
df_person.select(col('PRSN_AIRBAG_ID')).distinct().show()

[Stage 4:=====>                                                    (1 + 9) / 10]

+-----------------+
|   PRSN_AIRBAG_ID|
+-----------------+
|               NA|
|     NOT DEPLOYED|
|   DEPLOYED, SIDE|
|          UNKNOWN|
|   NOT APPLICABLE|
|DEPLOYED MULTIPLE|
|  DEPLOYED, FRONT|
|   DEPLOYED, REAR|
+-----------------+



In [10]:
df_person.groupBy(col('PRSN_AIRBAG_ID')).agg(count("*").alias('cnt')).select("PRSN_AIRBAG_ID", "cnt").show()

+-----------------+------+
|   PRSN_AIRBAG_ID|   cnt|
+-----------------+------+
|               NA|    19|
|     NOT DEPLOYED|118967|
|   DEPLOYED, SIDE|  2811|
|          UNKNOWN|  2291|
|   NOT APPLICABLE|  3910|
|DEPLOYED MULTIPLE|  7792|
|  DEPLOYED, FRONT| 21149|
|   DEPLOYED, REAR|    15|
+-----------------+------+



In [11]:
df_person.columns

['CRASH_ID',
 'UNIT_NBR',
 'PRSN_NBR',
 'PRSN_TYPE_ID',
 'PRSN_OCCPNT_POS_ID',
 'PRSN_INJRY_SEV_ID',
 'PRSN_AGE',
 'PRSN_ETHNICITY_ID',
 'PRSN_GNDR_ID',
 'PRSN_EJCT_ID',
 'PRSN_REST_ID',
 'PRSN_AIRBAG_ID',
 'PRSN_HELMET_ID',
 'PRSN_SOL_FL',
 'PRSN_ALC_SPEC_TYPE_ID',
 'PRSN_ALC_RSLT_ID',
 'PRSN_BAC_TEST_RSLT',
 'PRSN_DRG_SPEC_TYPE_ID',
 'PRSN_DRG_RSLT_ID',
 'DRVR_DRG_CAT_1_ID',
 'PRSN_DEATH_TIME',
 'INCAP_INJRY_CNT',
 'NONINCAP_INJRY_CNT',
 'POSS_INJRY_CNT',
 'NON_INJRY_CNT',
 'UNKN_INJRY_CNT',
 'TOT_INJRY_CNT',
 'DEATH_CNT',
 'DRVR_LIC_TYPE_ID',
 'DRVR_LIC_STATE_ID',
 'DRVR_LIC_CLS_ID',
 'DRVR_ZIP']

In [22]:
df_person = df_person.filter((col('PRSN_AIRBAG_ID').like("NOT DEPLOYED")) & (col('DEATH_CNT')>0))

In [23]:
# crash id count when airbag not deployed and death 
df_person.groupBy('CRASH_ID').agg(count("*").alias('cnt')).select('CRASH_ID', 'cnt').orderBy(col("cnt").desc()).show()

[Stage 38:============================>                            (5 + 5) / 10]

+--------+---+
|CRASH_ID|cnt|
+--------+---+
|15429998|  2|
|14848300|  1|
|14854567|  1|
|14920297|  1|
|14932526|  1|
|14940520|  1|
|14925518|  1|
|14909057|  1|
|15002687|  1|
|15005255|  1|
|15010732|  1|
|15014078|  1|
|15070158|  1|
|15043627|  1|
|15081189|  1|
|15114521|  1|
|15158717|  1|
|15149988|  1|
|15246994|  1|
|15208571|  1|
+--------+---+
only showing top 20 rows



In [16]:
df_temp = df_person.filter(col('CRASH_ID')=='15429998').toPandas()
display(df_temp)

,CRASH_ID,UNIT_NBR,PRSN_NBR,PRSN_TYPE_ID,PRSN_OCCPNT_POS_ID,PRSN_INJRY_SEV_ID,PRSN_AGE,PRSN_ETHNICITY_ID,PRSN_GNDR_ID,PRSN_EJCT_ID,PRSN_REST_ID,PRSN_AIRBAG_ID,PRSN_HELMET_ID,PRSN_SOL_FL,PRSN_ALC_SPEC_TYPE_ID,PRSN_ALC_RSLT_ID,PRSN_BAC_TEST_RSLT,PRSN_DRG_SPEC_TYPE_ID,PRSN_DRG_RSLT_ID,DRVR_DRG_CAT_1_ID,PRSN_DEATH_TIME,INCAP_INJRY_CNT,NONINCAP_INJRY_CNT,POSS_INJRY_CNT,NON_INJRY_CNT,UNKN_INJRY_CNT,TOT_INJRY_CNT,DEATH_CNT,DRVR_LIC_TYPE_ID,DRVR_LIC_STATE_ID,DRVR_LIC_CLS_ID,DRVR_ZIP
0,15429998,1,1,DRIVER,FRONT LEFT,KILLED,41,HISPANIC,MALE,NO,NONE,NOT DEPLOYED,NOT APPLICABLE,N,BLOOD,Positive,0.21,NONE,NOT APPLICABLE,NOT APPLICABLE,2025-01-20 00:50:00,0,0,0,0,0,0,1,DRIVER LICENSE,Texas,CLASS C,76513
1,15429998,2,1,DRIVER,FRONT LEFT,KILLED,31,BLACK,MALE,NO,UNKNOWN,NOT DEPLOYED,NOT APPLICABLE,N,BLOOD,NA,NA,BLOOD,UNKNOWN,UNKNOWN,2025-01-20 00:59:00,0,0,0,0,0,0,1,DRIVER LICENSE,Texas,CLASS C,76548


In [25]:
df_cars = df_person.select('CRASH_ID', 'DEATH_CNT').join(df_unit.select('CRASH_ID', 'VEH_MAKE_ID'), 'CRASH_ID', 'inner')

In [26]:
# distinct make ids
df_unit.select('VEH_MAKE_ID').distinct().show()

+-------------------+
|        VEH_MAKE_ID|
+-------------------+
|AMERICAN IRON HORSE|
|              ACURA|
|              WHITE|
|             PIERCE|
|            PORSCHE|
|           WHITEGMC|
|       FREIGHTLINER|
|              BUELL|
|           STERLING|
|            HYUNDAI|
|      INTERNATIONAL|
|               FIAT|
|                 NA|
|             TOYOTA|
|                RAM|
|             SUBARU|
|             NISSAN|
|     IC CORPORATION|
|            BENTLEY|
|             SUZUKI|
+-------------------+
only showing top 20 rows



In [29]:
df_cars.groupBy('VEH_MAKE_ID').agg(sum('DEATH_CNT').alias('death_cnt')).select('VEH_MAKE_ID', 'death_cnt').orderBy(col('death_cnt').desc()).show()

+-------------+---------+
|  VEH_MAKE_ID|death_cnt|
+-------------+---------+
|    CHEVROLET|       17|
|         FORD|       12|
|           NA|        7|
|        DODGE|        7|
| FREIGHTLINER|        6|
|       NISSAN|        5|
|        HONDA|        4|
|          GMC|        4|
|         JEEP|        3|
|     CADILLAC|        3|
|     INFINITI|        2|
|      PONTIAC|        2|
|    PETERBILT|        2|
|     KENWORTH|        2|
|         HINO|        2|
|   VOLKSWAGEN|        2|
|INTERNATIONAL|        2|
|      UNKNOWN|        2|
|     CHRYSLER|        2|
|         MACK|        1|
+-------------+---------+
only showing top 20 rows



In [30]:
df_make = spark.read.csv('../output/anaysis/analysis_3.csv', header=True)

In [31]:
df_make.show()

+------------+
| VEH_MAKE_ID|
+------------+
|   CHEVROLET|
|        FORD|
|          NA|
|       DODGE|
|FREIGHTLINER|
+------------+



# analysis 4

In [77]:
df_person = spark.read.csv(data_dir+'Primary_Person_use.csv', header=True, inferSchema=True)

In [33]:
df_person.columns

['CRASH_ID',
 'UNIT_NBR',
 'PRSN_NBR',
 'PRSN_TYPE_ID',
 'PRSN_OCCPNT_POS_ID',
 'PRSN_INJRY_SEV_ID',
 'PRSN_AGE',
 'PRSN_ETHNICITY_ID',
 'PRSN_GNDR_ID',
 'PRSN_EJCT_ID',
 'PRSN_REST_ID',
 'PRSN_AIRBAG_ID',
 'PRSN_HELMET_ID',
 'PRSN_SOL_FL',
 'PRSN_ALC_SPEC_TYPE_ID',
 'PRSN_ALC_RSLT_ID',
 'PRSN_BAC_TEST_RSLT',
 'PRSN_DRG_SPEC_TYPE_ID',
 'PRSN_DRG_RSLT_ID',
 'DRVR_DRG_CAT_1_ID',
 'PRSN_DEATH_TIME',
 'INCAP_INJRY_CNT',
 'NONINCAP_INJRY_CNT',
 'POSS_INJRY_CNT',
 'NON_INJRY_CNT',
 'UNKN_INJRY_CNT',
 'TOT_INJRY_CNT',
 'DEATH_CNT',
 'DRVR_LIC_TYPE_ID',
 'DRVR_LIC_STATE_ID',
 'DRVR_LIC_CLS_ID',
 'DRVR_ZIP']

In [39]:
# count of driving license
df_person.groupby('DRVR_LIC_TYPE_ID').agg(count("*").alias('cnt')).select('DRVR_LIC_TYPE_ID', 'cnt').show()

+--------------------+------+
|    DRVR_LIC_TYPE_ID|   cnt|
+--------------------+------+
|                  NA|   866|
|COMMERCIAL DRIVER...|  7204|
|             ID CARD| 11924|
|             UNKNOWN|  2268|
|        OCCUPATIONAL|   124|
|          UNLICENSED|  8988|
|               OTHER|   354|
|      DRIVER LICENSE|125226|
+--------------------+------+



In [40]:
# types of people involved in the crash 
df_person.select('PRSN_TYPE_ID').distinct().show()

+--------------------+
|        PRSN_TYPE_ID|
+--------------------+
|          PEDESTRIAN|
|DRIVER OF MOTORCY...|
|  PASSENGER/OCCUPANT|
|              DRIVER|
|             UNKNOWN|
|OTHER (EXPLAIN IN...|
|        PEDALCYCLIST|
+--------------------+



In [43]:
df_person.filter(col('PRSN_TYPE_ID').like("DRIVER")).select("*").show()

+--------+--------+--------+------------+------------------+--------------------+--------+-----------------+------------+------------+-------------------+-----------------+--------------+-----------+---------------------+----------------+------------------+---------------------+----------------+-----------------+---------------+---------------+------------------+--------------+-------------+--------------+-------------+---------+--------------------+-----------------+------------------+--------+
|CRASH_ID|UNIT_NBR|PRSN_NBR|PRSN_TYPE_ID|PRSN_OCCPNT_POS_ID|   PRSN_INJRY_SEV_ID|PRSN_AGE|PRSN_ETHNICITY_ID|PRSN_GNDR_ID|PRSN_EJCT_ID|       PRSN_REST_ID|   PRSN_AIRBAG_ID|PRSN_HELMET_ID|PRSN_SOL_FL|PRSN_ALC_SPEC_TYPE_ID|PRSN_ALC_RSLT_ID|PRSN_BAC_TEST_RSLT|PRSN_DRG_SPEC_TYPE_ID|PRSN_DRG_RSLT_ID|DRVR_DRG_CAT_1_ID|PRSN_DEATH_TIME|INCAP_INJRY_CNT|NONINCAP_INJRY_CNT|POSS_INJRY_CNT|NON_INJRY_CNT|UNKN_INJRY_CNT|TOT_INJRY_CNT|DEATH_CNT|    DRVR_LIC_TYPE_ID|DRVR_LIC_STATE_ID|   DRVR_LIC_CLS_ID|DRVR_ZIP

In [79]:
df_unit = spark.read.csv(data_dir+'Units_use.csv', header=True, inferSchema=True)

In [52]:
df_unit.select('VEH_HNR_FL').distinct().show()

+----------+
|VEH_HNR_FL|
+----------+
|         Y|
|         N|
|      NULL|
+----------+



In [55]:
df_unit.filter(col('VEH_HNR_FL')=='Y').groupBy('VIN').agg(count("*").alias('cnt')).select('VIN', 'cnt').orderBy(col('cnt').desc()).show()

+-----------------+----+
|              VIN| cnt|
+-----------------+----+
|             NULL|1083|
|          UNKNOWN|  60|
|1FTRW07L23KA57377|   6|
|1FTRX17W3XKB38908|   5|
|              UNK|   5|
|1XKDDB9XX1R873693|   4|
|1GTEK14K8NZ515381|   3|
|1FTRX17L1XKB99454|   3|
|1D7HA18N78S580746|   3|
|1G1BN52P4RR107166|   3|
|1FTHE24HXTHA78498|   2|
|1FUJGLCK19LAE8804|   2|
|1GCDS136258108226|   2|
|1FDKF37G9SNB15175|   2|
|4T3ZK3BB4DU056381|   2|
|2PCH33495FC712865|   2|
|1FMCU0EG2AKB61443|   2|
|1FUYDSEB6YLG17700|   2|
|1G2JC14T8P7571232|   2|
|1C6RD6HT3CS313220|   2|
+-----------------+----+
only showing top 20 rows



In [60]:
df_temp = df_unit.filter(col('VIN').isNull()).limit(5).toPandas()
display(df_temp)

,CRASH_ID,UNIT_NBR,UNIT_DESC_ID,VEH_PARKED_FL,VEH_HNR_FL,VEH_LIC_STATE_ID,VIN,VEH_MOD_YEAR,VEH_COLOR_ID,VEH_MAKE_ID,VEH_MOD_ID,VEH_BODY_STYL_ID,EMER_RESPNDR_FL,OWNR_ZIP,FIN_RESP_PROOF_ID,FIN_RESP_TYPE_ID,VEH_DMAG_AREA_1_ID,VEH_DMAG_SCL_1_ID,FORCE_DIR_1_ID,VEH_DMAG_AREA_2_ID,VEH_DMAG_SCL_2_ID,FORCE_DIR_2_ID,VEH_INVENTORIED_FL,VEH_TRANSP_NAME,VEH_TRANSP_DEST,CONTRIB_FACTR_1_ID,CONTRIB_FACTR_2_ID,CONTRIB_FACTR_P1_ID,VEH_TRVL_DIR_ID,FIRST_HARM_EVT_INV_ID,INCAP_INJRY_CNT,NONINCAP_INJRY_CNT,POSS_INJRY_CNT,NON_INJRY_CNT,UNKN_INJRY_CNT,TOT_INJRY_CNT,DEATH_CNT
0,14838637,3,TOWED/TRAILER,None,None,NA,None,NA,NA,NA,NA,NA,None,78934,NA,NA,NA,NA,NA,NA,NA,NA,None,None,None,NONE,NA,NA,NOT APPLICABLE,N,0,0,0,0,0,0,0
1,14839343,1,MOTOR VEHICLE,N,N,UN,None,NA,WHI,NISSAN,TITAN,PICKUP,N,None,2,NA,NA,NA,NA,NA,NA,NA,N,NOT TOWED,NOT TOWED,NA,NA,CHANGED LANE WHEN UNSAFE,SOUTHEAST,N,0,0,0,1,0,0,0
2,14840537,1,MOTOR VEHICLE,N,Y,UN,None,NA,BLK,CHEVROLET,SILVERADO,PICKUP,N,None,NA,NA,FRONT END DAMAGE DISTRIBUTED IMPACT,DAMAGED 2,12,NA,NA,NA,N,DRIVER,FLED SCENE,NA,NA,UNDER INFLUENCE - ALCOHOL,EAST,Y,0,0,0,0,1,0,0
3,14840833,2,NON-CONTACT,N,N,UN,None,NA,99,UNKNOWN,UNKNOWN,SPORT UTILITY VEHICLE,N,None,2,NA,NA,NA,NA,NA,NA,NA,N,None,None,NONE,NA,NA,EAST,N,0,0,0,0,1,0,0
4,14840833,2,NON-CONTACT,N,N,UN,None,NA,99,UNKNOWN,UNKNOWN,SPORT UTILITY VEHICLE,N,None,2,NA,NA,NA,NA,NA,NA,NA,N,None,None,NONE,NA,NA,EAST,N,0,0,0,0,1,0,0


In [63]:
df_person = df_person.filter(~(col('DRVR_LIC_TYPE_ID')!='NA'))

In [70]:
df_person.count()

866

In [78]:
df_person = df_person.filter((col('DRVR_LIC_TYPE_ID')!='NA') & (col('DRVR_LIC_TYPE_ID')!='UNLICENSED'))

In [80]:
df_unit = df_unit.filter(col('VEH_HNR_FL')=='Y')

In [82]:
df_hnr = df_person.select('CRASH_ID').join(df_unit.select('CRASH_ID', 'VIN'), 'CRASH_ID', 'inner')

In [85]:
df_hnr = df_person.join(df_unit, 'CRASH_ID', 'inner')

In [86]:
df_hnr.show()

+--------+--------+--------+------------+------------------+-----------------+--------+-----------------+------------+------------+-------------------+--------------+--------------+-----------+---------------------+----------------+------------------+---------------------+----------------+-----------------+---------------+---------------+------------------+--------------+-------------+--------------+-------------+---------+--------------------+-----------------+---------------+--------+--------+-------------+-------------+----------+----------------+-----------------+------------+------------+------------+----------+--------------------+---------------+--------+-----------------+--------------------+--------------------+-----------------+--------------+--------------------+-----------------+--------------+------------------+--------------------+--------------------+--------------------+------------------+-------------------+---------------+---------------------+---------------+--------

In [97]:
df_hnr.groupBy('VIN').agg(count("*").alias('cnt')).select('VIN', 'cnt').orderBy(col('cnt').desc()).show(20)

+-----------------+----+
|              VIN| cnt|
+-----------------+----+
|             NULL|2830|
|          UNKNOWN| 141|
|              UNK|  16|
|1XKDDB9XX1R873693|  16|
|3AKJGLDR1ESFK5634|   8|
|           UNKOWN|   8|
|1FTRF12247KA94683|   8|
|1GNDV23E75D123730|   7|
|1FTNW20F6YEC08788|   7|
|JTDKB20U183304899|   7|
|3GCPCREC2FG268856|   7|
|1FM5K7D80FGB56074|   7|
|1FT7W2BT0DEA96110|   7|
|1XKYDP9X5GJ491142|   6|
|4V4NC9TH93N336888|   6|
|1XP5DB9X84D832284|   6|
|1FMRU17LXWLB55367|   6|
|3GNDA23D97S539445|   6|
|1FTRX17L1XKB99454|   6|
|1GKFC13C27R360197|   6|
+-----------------+----+
only showing top 20 rows



In [93]:
cnt = df_hnr.filter(col('VIN').isNull()).count()
cnt

2830

In [92]:
df_temp = df_unit.filter(col('CRASH_ID')=='15444266').toPandas()
display(df_temp)

,CRASH_ID,UNIT_NBR,UNIT_DESC_ID,VEH_PARKED_FL,VEH_HNR_FL,VEH_LIC_STATE_ID,VIN,VEH_MOD_YEAR,VEH_COLOR_ID,VEH_MAKE_ID,VEH_MOD_ID,VEH_BODY_STYL_ID,EMER_RESPNDR_FL,OWNR_ZIP,FIN_RESP_PROOF_ID,FIN_RESP_TYPE_ID,VEH_DMAG_AREA_1_ID,VEH_DMAG_SCL_1_ID,FORCE_DIR_1_ID,VEH_DMAG_AREA_2_ID,VEH_DMAG_SCL_2_ID,FORCE_DIR_2_ID,VEH_INVENTORIED_FL,VEH_TRANSP_NAME,VEH_TRANSP_DEST,CONTRIB_FACTR_1_ID,CONTRIB_FACTR_2_ID,CONTRIB_FACTR_P1_ID,VEH_TRVL_DIR_ID,FIRST_HARM_EVT_INV_ID,INCAP_INJRY_CNT,NONINCAP_INJRY_CNT,POSS_INJRY_CNT,NON_INJRY_CNT,UNKN_INJRY_CNT,TOT_INJRY_CNT,DEATH_CNT
0,15444266,1,MOTOR VEHICLE,N,Y,TX,1FMZU62E52ZB10981,2002,BLK,FORD,EXPLORER,SPORT UTILITY VEHICLE,N,75220,1,PROOF OF LIABILITY INSURANCE,DISTRIBUTED RIGHT SIDE DAMAGE PARALLEL IMPACT,DAMAGED 3,3,DISTRIBUTED LEFT SIDE DAMAGE PARALLEL IMPACT,DAMAGED 3,10,Y,CONTRACT WRECKER,POLICE AUTO POUND,DISREGARD STOP AND GO SIGNAL,FAILED TO YIELD ROW - EMERGENCY VEHICLE,NA,NORTH,Y,0,0,0,1,0,0,0
1,15444266,2,MOTOR VEHICLE,N,Y,TX,2T1CE22PX2C007941,2002,BLK,TOYOTA,CAMRY,"PASSENGER CAR, 4-DOOR",N,75243,1,PROOF OF LIABILITY INSURANCE,DISTRIBUTED LEFT SIDE DAMAGE PARALLEL IMPACT,DAMAGED 3,9,NA,NA,NA,N,None,None,DISREGARD STOP AND GO SIGNAL,NA,NA,SOUTH,Y,0,0,0,1,0,0,0
2,15444266,3,MOTOR VEHICLE,N,Y,TX,2HGES15134H586493,2004,GRY,HONDA,ACCORD,"PASSENGER CAR, 4-DOOR",N,76021,1,PROOF OF LIABILITY INSURANCE,LEFT FRONT QUARTER DAMAGE ANGULAR IMPACT,DAMAGED 2,10,NA,NA,NA,N,None,None,NONE,NA,NA,WEST,N,0,0,0,3,0,0,0
3,15444266,4,MOTOR VEHICLE,N,Y,UN,UNK,NA,WHI,UNITED EXPRESS LINE INC,UNKNOWN,"PASSENGER CAR, 4-DOOR",N,None,2,NA,DISTRIBUTED RIGHT SIDE DAMAGE PARALLEL IMPACT,DAMAGED 2,2,NA,NA,NA,N,None,None,DISREGARD STOP AND GO SIGNAL,NA,NA,SOUTH,N,0,0,0,0,1,0,0


In [110]:
df_hnr = df_hnr.withColumn('vin_', length(col('VIN')))

In [113]:
df_temp = df_hnr.filter(length(col('VIN')) < 17).select('vin').orderBy(col('vin_').asc()).toPandas()
display(df_temp)

,vin
0,0
1,0
2,0
3,0
4,0
5,0
6,UN
7,UN
8,UN
9,UNK
